In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import linregress
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB

from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, concatenate, Conv2D
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def shrink_array(array,size):
    
    ratio = float(len(array)) / float(size+1)
    res = []
    for i in range(size):
        res.append(np.mean(array[math.floor(i*ratio):math.ceil((i+2.0)*ratio)], axis = 0))
    return np.array(res)

train_data = pd.DataFrame.from_csv("../../data/database/train_data.csv")
test_data = pd.DataFrame.from_csv("../../data/database/test_data.csv")
f_df = pd.DataFrame.from_csv("../../data/gesture_feature_df.csv")

feature_cols = ['global_acc3','acc_12_square']

def get_feature_label(data, N):
    groups = data.groupby(['TagName','tester_id'])
    keys = groups.groups.keys()
    y = []
    X = []
    for k in keys:
        frame_feature = shrink_array(groups.get_group(k)[feature_cols].values, N)
        X.append(frame_feature)
        y.append(k[0])
    return np.array(X),np.array(y)

def get_feature_label_f(data, N):
    groups = data.groupby(['TagName','tester_id'])
    keys = groups.groups.keys()
    y = []
    X = []
    f = []
    for k in keys:
        frame_feature = shrink_array(groups.get_group(k)[feature_cols].values, N)
        X.append(frame_feature)
        y.append(k[0])
        f.append(f_df[(f_df.TagName == k[0]) & (f_df.tester_id == k[1])].d_change.values[0])
    return np.array(X),np.array(y),np.array(f)

In [3]:
ensemble_06 = True
if ensemble_06 is True:
    rf_clf = joblib.load('../train/binary_model.pkl') 

In [4]:
    N=14
    X_train, y_train = get_feature_label(train_data, N)
    x_test, y_test, f_test = get_feature_label_f(test_data, N)

In [5]:
    y = np.concatenate([y_train,y_test])
    tag_list = []
    for i in range(10):
        tag_list.append(['Tag'+str(i),i])
    for i in tag_list:
        tag_str = i[0]
        tag_int = i[1]
        y[y==tag_str] = tag_int
    y_categorical = to_categorical(y)
    
    y_train_cate = y_categorical[:len(y_train)]
    y_test_cate = y_categorical[len(y_train):]

In [6]:

input_val1 = Input(shape=X_train.shape[1:])

con1 = Conv1D(filters=30,kernel_size=5)(input_val1)
max_pooling_1d_1 = MaxPooling1D(pool_size=2, strides=None, padding='valid')(con1)
flat_1 = Flatten()(max_pooling_1d_1)
layer2 = Dense(128, activation='relu')(flat_1)
layer4 = Dense(32, activation='relu')(layer2)
predictions = Dense(y_categorical.shape[-1], activation='softmax')(layer4)

model = Model(inputs = input_val1, outputs=predictions)
#         print(model.summary())

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_his = model.fit(X_train, y_train_cate, batch_size=32, epochs=32, verbose = 0,
                      validation_data=(x_test, y_test_cate)
                     )  # starts training


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [8]:
X_train.shape

(830, 14, 2)

In [ ]:
g = tf.Graph()
with g.as_default():
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    graph_def = g.as_graph_def()
    tf.train.write_graph(graph_def, './tmp/', 'model_not_freeze.pb', as_text=False)

In [ ]:
saver = tf.train.Saver()
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    save_path = saver.save(sess, "./tmp/model.ckpt")

In [ ]:
model_json = model.to_json()
with open("../train/tmp/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./tmp/model_only_weight.h5")
model.save("./tmp/model_weight_structure.h5")

In [ ]:
export_dir = "./SavedModelBuilder/"

builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.Session(graph=tf.Graph()) as sess:

      builder.add_meta_graph_and_variables(sess,
                                       ["foo-tag"])

builder.save()